# Regression 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

In [156]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [157]:
housing = pd.read_csv("cal_housing_clean.csv")

In [158]:
housing.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [159]:
housing.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
housingMedianAge,20640.0,28.639486,12.585558,1.0000,18.0000,29.0000,37.00000,52.0000
totalRooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.00000,39320.0000
totalBedrooms,20640.0,537.898014,421.247906,1.0000,295.0000,435.0000,647.00000,6445.0000
population,20640.0,1425.476744,1132.462122,3.0000,787.0000,1166.0000,1725.00000,35682.0000
households,20640.0,499.539680,382.329753,1.0000,280.0000,409.0000,605.00000,6082.0000
medianIncome,20640.0,3.870671,1.899822,0.4999,2.5634,3.5348,4.74325,15.0001
medianHouseValue,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.00000,500001.0000


In [160]:
x_data = housing.drop(['medianHouseValue'],axis=1)

In [161]:
y_val = housing['medianHouseValue']

In [162]:
from sklearn.model_selection import train_test_split

In [163]:
X_train, X_test, Y_train, Y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [164]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [165]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [166]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [167]:
X_train.shape

(14448, 6)

In [168]:
Y_train = Y_train.values.reshape(len(Y_train),1)

### TENSORFLOW CODE

In [197]:
batch_size = 10 
epochs = 100
x_train = tf.placeholder(tf.float64,[batch_size,6],name='X_t')
y_train = tf.placeholder(tf.float64,[batch_size,1],name = 'Y_t')
W = tf.Variable(np.random.uniform(low=2.0,high=4.0,size=6).reshape(6,1),name='weight')

y_pred = tf.matmul(x_train,W,name="Predictions")
error = y_train - y_pred
mse = tf.reduce_mean(tf.square(error),name= 'mse')
rmse = tf.sqrt(mse,name = 'rmse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train = optimizer.minimize(rmse)

init = tf.global_variables_initializer()

In [198]:
with tf.Session() as sess:
    init.run()
    print(sess.run([W]))

[array([[3.51141687],
       [2.54692132],
       [2.28577998],
       [2.98831399],
       [2.18895302],
       [2.61548222]])]


In [199]:
with tf.Session() as sess:
    init.run()
    
    batches = int(len(X_train)/10)
    
    for k in range(epochs):
        
        error_min = []
        
        for i in range(batches):
            rand_int =  np.random.randint(len(X_train),size=batch_size)
            feed = {x_train : X_train[rand_int],y_train : Y_train[rand_int]}
            sess.run(train,feed_dict=feed)
            error_min.append(rmse.eval(feed))
            
        print("Epoch :",k,"RMSE:", np.mean(np.array(error_min)))
                                    
    final_w = sess.run([W])                             

Epoch : 0 RMSE: 231098.67036848096
Epoch : 1 RMSE: 232857.461405611
Epoch : 2 RMSE: 232248.46032622518
Epoch : 3 RMSE: 232518.53235358634
Epoch : 4 RMSE: 231335.36252711256
Epoch : 5 RMSE: 231631.60823987453
Epoch : 6 RMSE: 233443.10968537666
Epoch : 7 RMSE: 233363.72304824446
Epoch : 8 RMSE: 233199.20378607637
Epoch : 9 RMSE: 233687.89135448853
Epoch : 10 RMSE: 231389.5748649552
Epoch : 11 RMSE: 234855.5810910273
Epoch : 12 RMSE: 232518.6425215383
Epoch : 13 RMSE: 230684.62705894187
Epoch : 14 RMSE: 233064.0843992185
Epoch : 15 RMSE: 233492.21673641892
Epoch : 16 RMSE: 230136.10256774435
Epoch : 17 RMSE: 234889.54063673387
Epoch : 18 RMSE: 230231.32829656132
Epoch : 19 RMSE: 231621.9281502528
Epoch : 20 RMSE: 234252.63662236682
Epoch : 21 RMSE: 232077.99870063117
Epoch : 22 RMSE: 232672.6393525855
Epoch : 23 RMSE: 232581.60066728192
Epoch : 24 RMSE: 230636.18382747873
Epoch : 25 RMSE: 233935.35763639238
Epoch : 26 RMSE: 233096.18392144155
Epoch : 27 RMSE: 233894.6410730563
Epoch : 28 

In [200]:
final_w

[array([[74.59213695],
        [11.63688954],
        [13.22877865],
        [ 9.36250524],
        [13.02266709],
        [37.97738893]])]

In [201]:
final_pred = np.matmul(X_test,final_w[0])

In [202]:
from sklearn.metrics import mean_squared_error

In [213]:
print(np.sqrt(mean_squared_error(final_pred,Y_test)))

237976.90401488342
